Make a big pandas dataframe to hold all the metric values we're interested in, so then we can pass it around and plot it up any way we like

I guess depth in each filter for each DDF?



In [1]:
import glob
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pylab as plt
import os
%matplotlib inline

In [2]:
# I just coppied them over from hyak and sym linked them to this directory for ease of use
glance_dirs = glob.glob('./*_glance')
sci_dirs = glob.glob('./*_sci')

glance_dirs.sort()
sci_dirs.sort()


In [3]:
sci_dirs
sci_dirs = [name for name in sci_dirs if 'pause_sim' not in name]

In [4]:
glance_dirs
glance_dirs = [name for name in glance_dirs if 'pause_sim' not in name]

In [5]:
filenames = [name.replace('./', '').replace('_glance', '') for name in glance_dirs]
filenames = [name for name in filenames if 'pause_sim' not in name]

In [6]:
filenames

['baseline_nexp2_v1.7.1_10yrs',
 'pause2_pl183_v1.7_10yrs',
 'pause2_pl91_v1.7_10yrs',
 'pause3_v1.7_10yrs']

In [7]:
runNames =  [name[2:].replace('_v1.7_10yrs_glance','').replace('v1.7_10yrs_glance','').replace('v1.7_10yrs_glance','') for name in glance_dirs] 

runNames = [name.replace('_v1.7.1_10yrs_glance', '') for name in runNames]

versions = ['1.7.1']*len(glance_dirs) 

In [8]:
runNames

['baseline_nexp2', 'pause2_pl183', 'pause2_pl91', 'pause3']

In [9]:
# Set up the dataframe
df = pd.DataFrame(np.array([runNames, versions]).T, columns=['runName', 'version'])

In [10]:
# List of tupes with (metricName, summaryName) that we want to pull from glance dirs
mnamesname =[('parallax', 'best18k'),
            ('properMotion', 'best18k'),
            ('fO', 'fONv MedianNvis'),
            ('fO', 'fOArea')]

In [11]:
for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "%s") and summaryName= "%s";' % (names[0], names[1])
    for directory in glance_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals

In [12]:
# Add the median coadded depths in each filter
for filtername in 'ugrizy':
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "CoaddM5" and metricMetadata=" %s") and summaryName= "Median";' % (filtername)
    for directory in glance_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df['median_coadd_%s' % filtername] = column_vals


In [13]:
# List of tupes with (metricName, summaryName) that we want to pull from science dirs
mnamesname =[('SNIa_PrePeak', 'Mean'),
             ('SNIa_WellSampled', 'Mean'),
            ('Fast Microlensing', 'Mean'),
            ('Slow Microlensing', 'Mean'),
            ('KN_PrePeak', 'Mean'),
            ('GalaxyCountsMetric_extended', 'N Galaxies (all)'),
            ('TDEsPopMetric__some_color', 'Mean'),
             ('TDEsPopMetric__some_color_pu', 'Mean'), 
            ('WeakLensingNvisits fiveSigmaDepth, visitExposureTime%', 'Median')]

In [14]:
for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName like "%s") and summaryName= "%s";' % (names[0], names[1])
    for directory in sci_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals

In [15]:
# things that need some metadata
mnamesname =[('Nstars_no_crowding', 'Total N Stars, no crowding')]

for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "%s" and metricMetadata=" i") and summaryName= "%s";' % (names[0], names[1])
    for directory in sci_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            column_vals.append(np.max(result))
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals

    
mnamesname =[('ExgalM5_with_cuts', '3x2ptFoM')]

for names in mnamesname:
    column_vals = []
    sql = 'select summaryValue from summarystats where metricId = (select metricId from metrics where metricName = "%s" and metricMetadata="i band non-DD year 10") and summaryName= "%s";' % (names[0], names[1])
    for directory in sci_dirs:
        conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
        result = pd.read_sql(sql, conn).values.ravel()
        if np.size(result) > 1:
            ValueError
        else:
            try:
                column_vals.append(np.max(result))
            except:
                import pdb ;pdb.set_trace()
        conn.close()
    df[names[0]+'_'+names[1]] = column_vals


In [16]:
np.max(result)

423.1788401648663

In [17]:
# Pull the DDF depths from everywhere. 
filters = 'ugrizy'
ddf_names = ['DD:ELAISS1', 'DD:XMM-LSS', 'DD:ECDFS', 'DD:COSMOS', 'DD:EDFS', 'DD:EDFSa', 'DD:EDFSb']

for filtername in filters:
    for ddf_name in ddf_names:
        column_vals = []
        sql = 'select summaryValue from summarystats where summaryName="Median depth %s, %s"' % (ddf_name, filtername)
        for directory in sci_dirs:
            conn = sqlite3.connect(directory+'/resultsDb_sqlite.db')
            result = pd.read_sql(sql, conn).values.ravel()
            if np.size(result) > 1:
                ValueError
            else:
                column_vals.append(np.max(result))
            conn.close()
        df[ddf_name+','+filtername] = column_vals

In [18]:
# Now to pull things from the solar system results

# here's what we used last time:
#'3 pairs in 15 nights detection loss NEO H=16.0',
#        '3 pairs in 30 nights detection loss NEO H=22.0',
#       '3 pairs in 15 nights detection loss TNO H=4.0'

faint_neo = []
bright_neo = []
tno = []

for name in filenames:
    conn = sqlite3.connect('%s_granvik_5k/resultsDb_sqlite.db' % name)
    sql = 'select summaryValue from summarystats where summaryName="DifferentialCompleteness H = 16.000000" and '+\
        'metricId = (select metricId from metrics where metricName ="Discovery_N_Chances" and metricMetadata="NEO 3 pairs in 15 nights detection loss")'
    val = np.max(pd.read_sql(sql, conn).values.ravel())
    bright_neo.append(val)
    
    
    sql = 'select summaryValue from summarystats where summaryName="DifferentialCompleteness H = 22.000000" and '+\
        'metricId = (select metricId from metrics where metricName ="Discovery_N_Chances" and metricMetadata="NEO 3 pairs in 15 nights detection loss")'

    val = np.max(pd.read_sql(sql, conn).values.ravel())
    faint_neo.append(val)
    
    conn.close()
    conn = sqlite3.connect('%s_l7_5k/resultsDb_sqlite.db' % name)
    
    sql = 'select summaryValue from summarystats where summaryName="DifferentialCompleteness H = 6.000000" and '+\
          'metricId = (select metricId from metrics where metricName ="Discovery_N_Chances" and metricMetadata="TNO 3 pairs in 15 nights detection loss")'

    val = np.max(pd.read_sql(sql, conn).values.ravel())
    tno.append(val)
    conn.close()
    
df['NEO bright'] = bright_neo
df['NEO faint'] = faint_neo 
df['TNO'] = tno

In [19]:
print('%s_granvik_5k/resultsDb_sqlite.db' % name)

pause3_v1.7_10yrs_granvik_5k/resultsDb_sqlite.db


In [20]:
df['NEO faint']

0    0.3942
1    0.3970
2    0.3998
3    0.4008
Name: NEO faint, dtype: float64

In [21]:
df['NEO bright']

0    0.9102
1    0.9152
2    0.9120
3    0.9132
Name: NEO bright, dtype: float64

In [22]:
runNames[0]

'baseline_nexp2'

In [23]:
plt.plot(df['median_coadd_g'])

In [24]:
df.columns

Index(['runName', 'version', 'parallax_best18k', 'properMotion_best18k',
       'fO_fONv MedianNvis', 'fO_fOArea', 'median_coadd_u', 'median_coadd_g',
       'median_coadd_r', 'median_coadd_i', 'median_coadd_z', 'median_coadd_y',
       'SNIa_PrePeak_Mean', 'SNIa_WellSampled_Mean', 'Fast Microlensing_Mean',
       'Slow Microlensing_Mean', 'KN_PrePeak_Mean',
       'GalaxyCountsMetric_extended_N Galaxies (all)',
       'TDEsPopMetric__some_color_Mean', 'TDEsPopMetric__some_color_pu_Mean',
       'WeakLensingNvisits fiveSigmaDepth, visitExposureTime%_Median',
       'Nstars_no_crowding_Total N Stars, no crowding',
       'ExgalM5_with_cuts_3x2ptFoM', 'DD:ELAISS1,u', 'DD:XMM-LSS,u',
       'DD:ECDFS,u', 'DD:COSMOS,u', 'DD:EDFS,u', 'DD:EDFSa,u', 'DD:EDFSb,u',
       'DD:ELAISS1,g', 'DD:XMM-LSS,g', 'DD:ECDFS,g', 'DD:COSMOS,g',
       'DD:EDFS,g', 'DD:EDFSa,g', 'DD:EDFSb,g', 'DD:ELAISS1,r', 'DD:XMM-LSS,r',
       'DD:ECDFS,r', 'DD:COSMOS,r', 'DD:EDFS,r', 'DD:EDFSa,r', 'DD:EDFSb,r',
       '

In [25]:
# pickle would be much smaller, but I guess can zip after if we want to
df.to_hdf('combined_maf_dataframe.hdf', 'maf')
# restore with:
# df = pd.read_hdf('combined_maf_dataframe.hdf', 'maf')